In [1]:
import sys
import torch
from datetime import date
import numpy

sys.path.append('../')
from src.fakenews import LabelFeature_Experiment

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"

Using backend: pytorch[22:00:54] /opt/dgl/src/runtime/tensordispatch.cc:43: TensorDispatcher: dlopen failed: /home/csgpu/anaconda3/envs/tf-gpu-cuda10/lib/python3.9/site-packages/dgl/tensoradapter/pytorch/libtensoradapter_pytorch_1.11.0.so: cannot open shared object file: No such file or directory



#### GENERATE EXPERIMENT (READ DATA & CONVERT TENSOR FORMAT & DETERMINE MASKS)

In [2]:
feature_names = ['alexa_rank','daily_pageviews_per_visitor','daily_time_on_site','total_sites_linking_in','bounce_rate']
syn_labels = [f'x{i}' for i in range(32)]
empty_list =  []
label_features = [f'labelf{i}' for i in range(4)] 

# feature_names = feature_names + label_features  # if you want to use labelfeature approach
# you should also move to batch training

coeffs = { 'model_type': 'gcn',
          'epoch':1000,
          'num_layers':4,
          'dim':128,
          'use_batch':False,
          'outer_batch_size':5000,
          'fan_out':[-1,25,10],
          'inner_batch_size':40,
          'train_percentage':0.8,
          'seed':0,
          'experiment_id':4,
          'gpu_id':0,
          'extra':'s1',
          'labelfeature_names':label_features,
          # 'labelfeature_names':['feat_pred_ben_def','feat_pred_mal_def'],
        #   'use_syn':True,
          'syn_labels':syn_labels,
          'syn_file':'../data/fakenews_xavier.csv'
}

ds_names = ["acl2020", "emnlp2018"]
tasks = ['fact', 'bias']
model_names = ['gcn', 'sagesup', 'sageunsup']
level = 3
ds_name = ds_names[1]
task = tasks[0]
model_name = model_names[0]

nodes_file = '../data/features_{}_level{}.txt4'.format(ds_name, level) 
edges_file = '../data/edges_{}_level{}.txt2'.format(ds_name, level)

model_file = f"""../model/fakenews_{coeffs['model_type']}.pkl"""      

#### TRAIN

In [3]:
exp = LabelFeature_Experiment(nodes_file, edges_file, feature_names, **coeffs)

model = exp.train(model_file, feature_names)
# model = exp.train_batches()
exp.save_model(model,  model_file)
# torch.save(model.state_dict(), model_file)

Train tensor(848) Test tensor(212)
Data converted to undirected: True
Data(x=[78428, 5], edge_index=[2, 232529], y=[78428], train_mask=[78428], test_mask=[78428], validation_mask=[78428])
Data converted to undirected: True
Data(x=[78428, 5], edge_index=[2, 232529], y=[78428], train_mask=[78428], test_mask=[78428], validation_mask=[78428])


RuntimeError: Error(s) in loading state_dict for Gcn:
	size mismatch for conv1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for conv1.lin.weight: copying a param with shape torch.Size([128, 5]) from checkpoint, the shape in current model is torch.Size([64, 5]).
	size mismatch for gcs.0.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for gcs.0.lin.weight: copying a param with shape torch.Size([128, 128]) from checkpoint, the shape in current model is torch.Size([64, 64]).
	size mismatch for gcs.1.bias: copying a param with shape torch.Size([128]) from checkpoint, the shape in current model is torch.Size([64]).
	size mismatch for gcs.1.lin.weight: copying a param with shape torch.Size([128, 128]) from checkpoint, the shape in current model is torch.Size([64, 64]).
	size mismatch for conv2.lin.weight: copying a param with 

#### TEST

In [ ]:
# model_file = f'./model/explainer_gcn_noavglf0_3_48_{date.today()}.pkl'       
accs = exp.test(model_file, feature_names, raw_directory='../results/raw_results/')

print('Test Accuracy: {:.4f}'.format(accs[0]), '\tF1: {:.4f}'.format(accs[1]), '\tAUC: {:.4f}'.format(accs[2]) )
accs

In [ ]:
emb = exp.test3(model_file, feature_names, raw_directory='../results/raw_results/')

In [ ]:
import pandas as pd
nodes_df = pd.read_csv(nodes_file)

In [ ]:
nodes_df.head(2)

In [ ]:
nodes_df.shape

In [ ]:
emb.shape

In [ ]:
emb_dict = dict(zip(nodes_df.node.tolist(), emb.tolist()))

In [ ]:
import json
emb_filename = "../data/{}_{}_level{}_{}.json".format(ds_name, task, level, model_name)
print(emb_filename)
with open(emb_filename, 'w') as emb_file:
    json.dump(emb_dict, emb_file)

In [ ]:
emb_dict['bayoubuzz.com']